In [1]:
import numpy as np
import pandas as pd
import warnings
import os
from tqdm import tqdm
from sklearn import preprocessing, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from sklearn.utils import shuffle
import joblib

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [2]:
seed = 2020

In [3]:
df_feature = pd.read_pickle('./temp/part1_feature.plk')
df_order = pd.read_pickle('./temp/order.plk')
df_feature = df_feature.merge(
    df_order[['tracking_id','assigned_time']], how='left')
df_feature.head(10)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,level,speed,max_load,assigned_time
0,117052073,1,2100074527716017758,121.455805,39.108616,PICKUP,1580528357,20200201,train,1.0,202002011170520731,2780,1580528211,2100074548913145283,PICKUP,121.441945,39.102244,121.439220,39.100178,532.0,2100074527716017758PICKUP,正常天气,1112f462a41e1dc3a774005086d11daa,ca2a062e0790356c72caa2eb909eeb2e,1580530500,1580527009,3,4.643877,11,1580527356
1,100681730,0,2100074530361804536,121.598811,39.149256,DELIVERY,1580526920,20200201,train,0.0,202002011006817300,1476,1580525615,2100074530361804536,PICKUP,121.601105,39.137024,121.584554,39.116471,4184.0,2100074530361804536DELIVERY,正常天气,e5059162349d42fe9e69450c1b42a33e,8944ec8db309614c49fc787d3ba12f44,1580526900,1580469230,2,4.489853,8,1580523845
2,117054417,0,2100074532411639390,121.448985,39.095307,PICKUP,1580525460,20200201,train,1.0,202002011170544170,2796,1580525458,2100074546017437278,PICKUP,121.439220,39.100178,121.439220,39.100178,1.0,2100074532411639390PICKUP,正常天气,0ab823ac03cbcae5000924eaf1ea7e12,ca2a062e0790356c72caa2eb909eeb2e,1580526900,1580472595,3,5.228923,11,1580524836
3,103645298,0,2100074532493362936,121.600431,39.137425,DELIVERY,1580524836,20200201,train,0.0,202002011036452980,1811,1580523872,2100074542979057400,PICKUP,121.601105,39.137024,121.619324,39.129741,2646.0,2100074532493362936DELIVERY,正常天气,5ef6c879fa914a0922fcc5e49f46077b,8944ec8db309614c49fc787d3ba12f44,1580525100,1580521892,3,4.371049,9,1580522646
4,10401287,0,2100074533161764858,121.569268,39.104077,PICKUP,1580521211,20200201,train,1.0,20200201104012870,673,1580521209,2100074542338410490,PICKUP,121.575545,39.103083,121.575545,39.103083,1.0,2100074533161764858PICKUP,正常天气,6a3dcff2d7fd7a5a3ba955859494bc1c,61be8c5f24588a313c738cc8e68f60a5,1580523300,1580516006,3,4.768473,7,1580520725
5,118916297,0,2100074533397890719,121.977101,39.595172,PICKUP,1580523014,20200201,train,0.0,202002011189162970,3610,1580522123,2100074545051993759,PICKUP,121.960521,39.588041,121.960521,39.588041,1.0,2100074533397890719PICKUP,正常天气,090290b5354073fa64a3266d81c9d09f,d6265126edc531b3cb13a00ed9ccd6bd,1580525100,1580475782,3,4.685788,9,1580522509
6,101915922,1,2100074533420893896,121.690685,39.279492,PICKUP,1580525730,20200201,train,0.0,202002011019159221,1608,1580524864,2100074545447667458,DELIVERY,121.674354,39.272571,121.677846,39.280096,1061.0,2100074533420893896PICKUP,正常天气,dd32ff9b2aa697b0792e4e27eaa9d904,ac5cd13db63c15230b57b3d1fffbdbf2,1580526900,1580476426,3,4.736526,8,1580523899
7,118691289,0,2100074533674616654,121.431624,39.189955,DELIVERY,1580524738,20200201,train,0.0,202002011186912890,3213,1580523464,2100074533674616654,PICKUP,121.419881,39.185003,121.398254,39.169541,3305.0,2100074533674616654DELIVERY,正常天气,f25cbfe835006fd049a069c952c5ee8a,52609cbee4c8444396b944529cd0856d,1580525100,1580474743,3,4.725414,10,1580522494
8,10290949,0,2100074533781079886,121.422817,39.184187,DELIVERY,1580522171,20200201,train,0.0,20200201102909490,388,1580521706,2100074539616995150,PICKUP,121.419881,39.185003,121.418574,39.190101,1702.0,2100074533781079886DELIVERY,正常天气,f2760ca7214e0081158a43d0d20c8245,52609cbee4c8444396b944529cd0856d,1580523300,1580520790,0,4.699998,9,1580521174
9,118662905,0,2100074534060164673,121.543779,39.081136,DELIVERY,1580523911,20200201,train,1.0,202002011186629050,3139,1580523617,2100074543608104054,DELIVERY,121.547704,39.087431,121.547034,39.087641,137.0,2100074534060164673DELIVERY,正常天气,172cd36588e2214177d25a13215e9d51,ff4dc09a24956989e98b205b8af41697,1580525100,1580476524,3,4.759711,10,1580523136


In [4]:
df_feature.shape

(150060, 30)

In [5]:

#可以不运行

df_feature.loc[df_feature.weather_grade=='正常天气','weather_grade'] = 5
df_feature.loc[df_feature.weather_grade=='轻微恶劣天气','weather_grade'] = 10
df_feature.loc[df_feature.weather_grade=='恶劣天气','weather_grade'] = 15
df_feature.loc[df_feature.weather_grade=='极恶劣天气','weather_grade'] = 20
df_feature.loc[df_feature.weather_grade=='罕见恶劣天气','weather_grade'] = 25
df_feature['weather_grade']=df_feature['weather_grade'].astype('float')
df_feature.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,level,speed,max_load,assigned_time
0,117052073,1,2100074527716017758,121.455805,39.108616,PICKUP,1580528357,20200201,train,1.0,202002011170520731,2780,1580528211,2100074548913145283,PICKUP,121.441945,39.102244,121.439220,39.100178,532.0,2100074527716017758PICKUP,5.0,1112f462a41e1dc3a774005086d11daa,ca2a062e0790356c72caa2eb909eeb2e,1580530500,1580527009,3,4.643877,11,1580527356
1,100681730,0,2100074530361804536,121.598811,39.149256,DELIVERY,1580526920,20200201,train,0.0,202002011006817300,1476,1580525615,2100074530361804536,PICKUP,121.601105,39.137024,121.584554,39.116471,4184.0,2100074530361804536DELIVERY,5.0,e5059162349d42fe9e69450c1b42a33e,8944ec8db309614c49fc787d3ba12f44,1580526900,1580469230,2,4.489853,8,1580523845
2,117054417,0,2100074532411639390,121.448985,39.095307,PICKUP,1580525460,20200201,train,1.0,202002011170544170,2796,1580525458,2100074546017437278,PICKUP,121.439220,39.100178,121.439220,39.100178,1.0,2100074532411639390PICKUP,5.0,0ab823ac03cbcae5000924eaf1ea7e12,ca2a062e0790356c72caa2eb909eeb2e,1580526900,1580472595,3,5.228923,11,1580524836
3,103645298,0,2100074532493362936,121.600431,39.137425,DELIVERY,1580524836,20200201,train,0.0,202002011036452980,1811,1580523872,2100074542979057400,PICKUP,121.601105,39.137024,121.619324,39.129741,2646.0,2100074532493362936DELIVERY,5.0,5ef6c879fa914a0922fcc5e49f46077b,8944ec8db309614c49fc787d3ba12f44,1580525100,1580521892,3,4.371049,9,1580522646
4,10401287,0,2100074533161764858,121.569268,39.104077,PICKUP,1580521211,20200201,train,1.0,20200201104012870,673,1580521209,2100074542338410490,PICKUP,121.575545,39.103083,121.575545,39.103083,1.0,2100074533161764858PICKUP,5.0,6a3dcff2d7fd7a5a3ba955859494bc1c,61be8c5f24588a313c738cc8e68f60a5,1580523300,1580516006,3,4.768473,7,1580520725


In [6]:
max_min_scaler = lambda x : ((x-np.min(x))/(np.max(x)-np.min(x))+0.1)
df_temp1 = df_feature[['speed']].apply(max_min_scaler)
df_feature = df_feature.drop("speed", axis=1)
df_feature["speed"] = df_temp1

In [7]:
df_temp2 = df_feature[['max_load']].apply(max_min_scaler)
df_feature = df_feature.drop("max_load", axis=1)
df_feature["max_load"] = df_temp2

In [8]:
df_temp4 = df_feature[['level']].apply(max_min_scaler)
df_feature = df_feature.drop("level", axis=1)
df_feature["level"] = df_temp4

In [9]:
df_feature['target_MHD'] = abs(df_feature['source_lat']-df_feature['target_lat']) + abs(df_feature['source_lng']-df_feature['target_lng'])# 加入曼哈顿距离

In [10]:
df_temp5 = df_feature[['target_MHD']].apply(max_min_scaler)
df_feature = df_feature.drop("target_MHD", axis=1)
df_feature["target_MHD"] = df_temp5

In [11]:
# sub_columns = ['courier_id', 'wave_index', 'tracking_id',
#                'courier_wave_start_lng', 'courier_wave_start_lat', 'action_type', 'expect_time']

In [12]:
#df_feature['distance2_3'] = df_feature['distance2_3'].astype('int')
for f in df_feature.select_dtypes('object'):
    if f not in ['date', 'type']:
        print(f)
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

action_type
group
last_action_type
TA
aoi_id
shop_id


In [13]:
df_feature.head(10)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,assigned_time,speed,max_load,level,target_MHD
0,117052073,1,2100074527716017758,121.455805,39.108616,1,1580528357,20200201,train,1.0,692,2780,1580528211,2100074548913145283,1,121.441945,39.102244,121.439220,39.100178,532.0,0,5.0,1853,8358,1580530500,1580527009,1580527356,0.515605,0.655556,1.100000,0.154234
1,100681730,0,2100074530361804536,121.598811,39.149256,0,1580526920,20200201,train,0.0,40,1476,1580525615,2100074530361804536,1,121.601105,39.137024,121.584554,39.116471,4184.0,1,5.0,24806,5680,1580526900,1580469230,1580523845,0.476457,0.488889,0.766667,0.520018
2,117054417,0,2100074532411639390,121.448985,39.095307,1,1580525460,20200201,train,1.0,695,2796,1580525458,2100074546017437278,1,121.439220,39.100178,121.439220,39.100178,1.0,2,5.0,1130,8358,1580526900,1580472595,1580524836,0.664306,0.655556,1.100000,0.100000
3,103645298,0,2100074532493362936,121.600431,39.137425,0,1580524836,20200201,train,0.0,241,1811,1580523872,2100074542979057400,1,121.601105,39.137024,121.619324,39.129741,2646.0,3,5.0,10207,5680,1580525100,1580521892,1580522646,0.446260,0.544444,1.100000,0.388683
4,10401287,0,2100074533161764858,121.569268,39.104077,1,1580521211,20200201,train,1.0,279,673,1580521209,2100074542338410490,1,121.575545,39.103083,121.575545,39.103083,1.0,4,5.0,11454,4034,1580523300,1580516006,1580520725,0.547273,0.433333,1.100000,0.100000
5,118916297,0,2100074533397890719,121.977101,39.595172,1,1580523014,20200201,train,0.0,886,3610,1580522123,2100074545051993759,1,121.960521,39.588041,121.960521,39.588041,1.0,5,5.0,934,8874,1580525100,1580475782,1580522509,0.526257,0.544444,1.100000,0.100000
6,101915922,1,2100074533420893896,121.690685,39.279492,1,1580525730,20200201,train,0.0,110,1608,1580524864,2100074545447667458,0,121.674354,39.272571,121.677846,39.280096,1061.0,6,5.0,23968,7095,1580526900,1580476426,1580523899,0.539153,0.488889,1.100000,0.224713
7,118691289,0,2100074533674616654,121.431624,39.189955,0,1580524738,20200201,train,0.0,789,3213,1580523464,2100074533674616654,1,121.419881,39.185003,121.398254,39.169541,3305.0,7,5.0,26281,3391,1580525100,1580474743,1580522494,0.536329,0.600000,1.100000,0.519849
8,10290949,0,2100074533781079886,121.422817,39.184187,0,1580522171,20200201,train,0.0,175,388,1580521706,2100074539616995150,1,121.419881,39.185003,121.418574,39.190101,1702.0,8,5.0,26291,3391,1580523300,1580520790,1580521174,0.529869,0.544444,0.100000,0.172505
9,118662905,0,2100074534060164673,121.543779,39.081136,0,1580523911,20200201,train,1.0,772,3139,1580523617,2100074543608104054,0,121.547704,39.087431,121.547034,39.087641,137.0,9,5.0,2487,10592,1580525100,1580476524,1580523136,0.545046,0.600000,1.100000,0.109962


In [14]:
df_feature['promise_dif'] = df_feature['promise_deliver_time'] - df_feature['current_time']
df_feature['estimate_dif'] = df_feature['estimate_pick_time'] - df_feature['current_time']
df_feature['Restime_dif'] = df_feature['promise_deliver_time'] - df_feature['estimate_pick_time']
df_feature['assign_dif'] = df_feature['assigned_time'] - df_feature['current_time']
#df_feature['promise_assign_dif'] = df_feature['promise_deliver_time'] - df_feature['assigned_time']
#df_feature['estimate_assign_dif'] = df_feature['estimate_pick_time'] - df_feature['assigned_time']

In [15]:
df_feature.to_pickle('./temp/part1_speicial_feature.plk')
df_feature.head(10)

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,TA,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,assigned_time,speed,max_load,level,target_MHD,promise_dif,estimate_dif,Restime_dif,assign_dif
0,117052073,1,2100074527716017758,121.455805,39.108616,1,1580528357,20200201,train,1.0,692,2780,1580528211,2100074548913145283,1,121.441945,39.102244,121.439220,39.100178,532.0,0,5.0,1853,8358,1580530500,1580527009,1580527356,0.515605,0.655556,1.100000,0.154234,2289,-1202,3491,-855
1,100681730,0,2100074530361804536,121.598811,39.149256,0,1580526920,20200201,train,0.0,40,1476,1580525615,2100074530361804536,1,121.601105,39.137024,121.584554,39.116471,4184.0,1,5.0,24806,5680,1580526900,1580469230,1580523845,0.476457,0.488889,0.766667,0.520018,1285,-56385,57670,-1770
2,117054417,0,2100074532411639390,121.448985,39.095307,1,1580525460,20200201,train,1.0,695,2796,1580525458,2100074546017437278,1,121.439220,39.100178,121.439220,39.100178,1.0,2,5.0,1130,8358,1580526900,1580472595,1580524836,0.664306,0.655556,1.100000,0.100000,1442,-52863,54305,-622
3,103645298,0,2100074532493362936,121.600431,39.137425,0,1580524836,20200201,train,0.0,241,1811,1580523872,2100074542979057400,1,121.601105,39.137024,121.619324,39.129741,2646.0,3,5.0,10207,5680,1580525100,1580521892,1580522646,0.446260,0.544444,1.100000,0.388683,1228,-1980,3208,-1226
4,10401287,0,2100074533161764858,121.569268,39.104077,1,1580521211,20200201,train,1.0,279,673,1580521209,2100074542338410490,1,121.575545,39.103083,121.575545,39.103083,1.0,4,5.0,11454,4034,1580523300,1580516006,1580520725,0.547273,0.433333,1.100000,0.100000,2091,-5203,7294,-484
5,118916297,0,2100074533397890719,121.977101,39.595172,1,1580523014,20200201,train,0.0,886,3610,1580522123,2100074545051993759,1,121.960521,39.588041,121.960521,39.588041,1.0,5,5.0,934,8874,1580525100,1580475782,1580522509,0.526257,0.544444,1.100000,0.100000,2977,-46341,49318,386
6,101915922,1,2100074533420893896,121.690685,39.279492,1,1580525730,20200201,train,0.0,110,1608,1580524864,2100074545447667458,0,121.674354,39.272571,121.677846,39.280096,1061.0,6,5.0,23968,7095,1580526900,1580476426,1580523899,0.539153,0.488889,1.100000,0.224713,2036,-48438,50474,-965
7,118691289,0,2100074533674616654,121.431624,39.189955,0,1580524738,20200201,train,0.0,789,3213,1580523464,2100074533674616654,1,121.419881,39.185003,121.398254,39.169541,3305.0,7,5.0,26281,3391,1580525100,1580474743,1580522494,0.536329,0.600000,1.100000,0.519849,1636,-48721,50357,-970
8,10290949,0,2100074533781079886,121.422817,39.184187,0,1580522171,20200201,train,0.0,175,388,1580521706,2100074539616995150,1,121.419881,39.185003,121.418574,39.190101,1702.0,8,5.0,26291,3391,1580523300,1580520790,1580521174,0.529869,0.544444,0.100000,0.172505,1594,-916,2510,-532
9,118662905,0,2100074534060164673,121.543779,39.081136,0,1580523911,20200201,train,1.0,772,3139,1580523617,2100074543608104054,0,121.547704,39.087431,121.547034,39.087641,137.0,9,5.0,2487,10592,1580525100,1580476524,1580523136,0.545046,0.600000,1.100000,0.109962,1483,-47093,48576,-481


In [16]:
df_test = df_feature[df_feature['type'] == 'test'].copy()
df_train = df_feature[df_feature['type'] == 'train'].copy()
df_train = shuffle(df_train, random_state=seed)

In [54]:
ycol = 'target' #,'target_MHD','estimate_dif','weather_grade''distance2_3','courier_wave_start_lng', 'courier_wave_start_lat','source_lng', 'source_lat'
# ,,'Restime_dif','target_MHD','estimate_dif','promise_dif','assign_dif'
feature_names = list(
    filter(lambda x: x not in [ycol,'assigned_time','level','id', 'wave_index', 'tracking_id', 'expect_time', 'date', 'type', 'group',
                               'shop_id', 'current_time_date','estimate_pick_time', 'promise_deliver_time','courier_wave_start_lng', 'courier_wave_start_lat','source_lng', 'source_lat'
                               ,'target_lng','target_lat', 'current_time','last_tracking_id','aoi_id' ,'courier_id', 'TA'], df_train.columns))

model = lgb.LGBMClassifier(num_leaves=16, #64
                           max_depth=4,  #10
                           learning_rate=0.03, #0.005
                           n_estimators=1000000, 
                           subsample=0.8,  #0.9
                           feature_fraction=0.8, #0.8
                           reg_alpha=0.5,    #0.5
                           reg_lambda=0.5,   #0.5
                           random_state=seed,  
                           metric=None
                           )


oof = []
prediction = df_test[['id', 'group']]
prediction['target'] = 0
df_importance_list = []
trainRound = 1
kfold = GroupKFold(n_splits=5)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol], df_train['group'])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]
    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=500,
                          eval_metric='auc',
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof = df_train.iloc[val_idx][['id', 'group', ycol]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)
    
    

    pred_test = lgb_model.predict_proba(
        df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['target'] += pred_test / 5

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)
    joblib.dump(lgb_model,'./model/lgbmodel' + str(trainRound) + '.pkl')
    trainRound = trainRound + 1
    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's auc: 0.833528	valid's auc: 0.828441
Early stopping, best iteration is:
[710]	train's auc: 0.835969	valid's auc: 0.82859

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's auc: 0.833333	valid's auc: 0.827228
Early stopping, best iteration is:
[520]	train's auc: 0.833621	valid's auc: 0.8273

Fold_3 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's auc: 0.833299	valid's auc: 0.828355
Early stopping, best iteration is:
[697]	train's auc: 0.835752	valid's auc: 0.828522

Fold_4 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's auc: 0.834564	valid's auc: 0.824571
Early stopping, best iteration is:
[926]	train's auc: 0.839792	valid's auc: 0.824982

Fold_5 Trainin

In [55]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,assign_dif,1745.2
1,promise_dif,1272.6
2,estimate_dif,1115.2
3,target_MHD,1103.8
4,Restime_dif,1103.6
5,grid_distance,1088.4
6,speed,928.8
7,last_action_type,568.8
8,max_load,401.6
9,action_type,355.6


In [56]:
def wave_label_func(group):
    target_list = group['target'].values.tolist()
    pred_list = group['pred'].values.tolist()
    max_index = pred_list.index(max(pred_list))
    if target_list[max_index] == 1:
        return 1
    else:
        return 0

In [57]:
    df_oof = pd.concat(oof)
    df_temp = df_oof.groupby(['group']).apply(wave_label_func).reset_index()
    df_temp.columns = ['group', 'label']
    acc = df_temp[df_temp['label'] == 1].shape[0] / df_temp.shape[0]
    print('acc:', acc)

acc: 0.8020563802260656


In [ ]:
def label_func(group):
    group = group.values.tolist()
    max_index = group.index(max(group))
    label = np.zeros(len(group))
    label[max_index] = 1
    return label


prediction['label'] = prediction.groupby(
    ['group'])['target'].transform(label_func)
sub_part1 = prediction[prediction['label'] == 1]
df_oof = df_oof[df_oof['target'] == 1]
next_action = pd.concat([df_oof[['id']], sub_part1[['id']]])
next_action.to_csv('./temp/next_action.csv', index=False)